In [1]:
import numpy as np
from skimage import io
from skimage import img_as_ubyte
from skimage import exposure
import os
from shutil import copy
from timeit import default_timer as timer

In [2]:
def make_spiral_grid(width, height):
    '''
    The high content microscope images a well in spiral shape.
    This function makes a spiral-shaped grid, that we can use to find the locations of the image.
    '''
    NORTH, S, W, E = (0, -1), (0, 1), (-1, 0), (1, 0) # directions
    turn_right = {NORTH: W, E: NORTH, S: E, W: S} # old -> new direction
    
    if width < 1 or height < 1:
        raise ValueError
    
    x, y = int(np.ceil(width/2)-1), height // 2 # start near the center
    dx, dy = S # initial direction
    matrix = [[None] * width for _ in range(height)]
    count = 0
    while True:
        matrix[y][x] = count # visit
        count += 1
        # try to turn right
        new_dx, new_dy = turn_right[dx,dy]
        new_x, new_y = x + new_dx, y + new_dy
        if (0 <= new_x < width and 0 <= new_y < height and
            matrix[new_y][new_x] is None): # can turn right
            x, y = new_x, new_y
            dx, dy = new_dx, new_dy
        else: # try to move straight
            x, y = x + dx, y + dy
            if not (0 <= x < width and 0 <= y < height):
                return np.asarray(matrix) # nowhere to go
            
def make_column_grid(width, height):
    '''
    The FIJI stitching algorithm can stitch images in a column-by-column grid.
    This function makes a column-shaped grid.
    '''
    return np.arange(width * height).reshape(height,width).transpose()

def convert_array_to_string(a):
    '''
    This function takes as input a 2D array a.
    It outputs the same array, but integers are strings formatted as '00', '01', etc.
    '''
    if len(a.shape) == 1:
        a = [a]
    a_as_str = []
    for row in a:
        str_row = []
        for num in row:
            str_row.append("%02d" % num)
        a_as_str.append(str_row)        

    return a_as_str

In [3]:
def prepare_dataset_for_stitching(target_folder, img_folder, well_rows, well_cols, w):
    '''
    This function combines DAPI and MEMBRITE channels of acquired images and stores them under the correct filename,
    so they can be used for stitching in imageJ.
    
    Inputs:
        target_folder: path to folder of the experiment. It is named with the experiment data (e.g. 2020-09-01)
        img_folder: name of folder with the raw images (e.g. 'AcquireOnly.V3_09-01-20_04;16;23')
        well_rows: list of well rows that were imaged, e.g. ['B', 'D']
        well_cols: list of well columns that were imaged, e.g. [2, 3, 4]
        w = number of images in the stitch, e.g. 8
    '''

    raw_img_folder = os.path.join(target_folder, img_folder)

    # Define a spiral-shaped grid
    spiral_grid = make_spiral_grid(w,w)
    column_grid = make_column_grid(w, w)
    column_grid = convert_array_to_string(column_grid)

    nrs = convert_array_to_string( np.arange(w ** 2) )[0]

    file_list = os.listdir( raw_img_folder )

    # get first part of file name (is the same for all files in folder)
    splitted_file = file_list[1].split('_')
    img_info = splitted_file[0] + '_' + splitted_file[1] + '_'

    for row in well_rows:
        for col in well_cols:

            col = "%02d" % col
            well_folder = os.path.join(target_folder, 'well '+row+col)
            tile_folder = os.path.join(well_folder, 'tiles')

            # Create new directory for this well (if it didn't exist already)
            if not os.path.exists( well_folder ):
                os.mkdir(  well_folder )
            if not os.path.exists( tile_folder ):
                os.mkdir( tile_folder )

            count = 0
            images = []
            for nr in nrs:

                fname_MEMBR = img_info + row + col + 'f' + nr + 'd1.TIFF'
                img_MEMBR = io.imread( os.path.join(raw_img_folder, fname_MEMBR) )

                fname_DAPI = img_info + row + col + 'f' + nr + 'd0.TIFF'
                img_DAPI = io.imread( os.path.join(raw_img_folder, fname_DAPI) )

                N,M = img_MEMBR.shape
                img = np.zeros((N,M,3))

                img[:,:,0] = img_MEMBR
                img[:,:,2] = img_DAPI

                r,c = np.where(spiral_grid == count)
                tile_index = column_grid[r[0]][c[0]]
                fname = 'tile_' + tile_index + '.tif'
                io.imsave(os.path.join(tile_folder, fname), img)
                images.append(img)
                count = count + 1

In [4]:
target_folder = 'Data\\WKS023\\2020-09-09'
img_folder = 'AcquireOnly.V3_09-09-20_04;28;26'
well_rows = ['B', 'D']
well_cols = [3,4,5]
w = 8 # number of stitched images

prepare_dataset_for_stitching(target_folder, img_folder, well_rows, well_cols, w)